In [1]:
####################################
########### LOAD PKGS ##############
####################################
%matplotlib tk
%autosave 180

#%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

import glob

from classify_utility import * 

Autosaving every 180 seconds


In [13]:
#################################################
########### MAKE DATA LOADER FUNCTION ###########
#################################################

# FUNCTION REQUIRED FOR TORCH (?) ALSO FOR RESNET fixed sizes
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #transforms.Normalize([0.46, 0.48, 0.51], [0.32, 0.32, 0.32])
    ])

# DATA LOADER AND RANDOMIZER FUNCTION
def make_trainloader(train_data, 
                     vals, 
                     batch_size,
                     randomize=True):
    
    # RANDOMIZE DATA EVERYTIME THIS IS CALLED
    if randomize:
        idx = np.random.choice(np.arange(vals.shape[0]),
                         vals.shape[0],replace=False)
        # REARANGE DATA BASED ON RANDOMIZATION FLAG
        train_data = train_data[idx]
        vals = vals[idx]
    else:
        idx = np.arange(vals.shape[0])
    

    # Compute number of batches
    n_batches = train_data.shape[0]//batch_size

    # make trainign data plus labels
    data_train = []
    vals_train = []
    for k in range(0,n_batches*batch_size,batch_size):
        data_train.append(train_data[k:k+batch_size])
        vals_train.append(vals[k:k+batch_size])

    # 
    #print ("# batches: ", n_batches)
        
    # RATIO OF DATA SPLIT BETWEEN TRAIN AND TEST
    split = 0.8
    
    trainloader = zip(data_train[:int(len(data_train)*split)],
                      vals_train[:int(len(data_train)*split)])
    
    testloader = zip(data_train[int(len(data_train)*split):],
                      vals_train[int(len(data_train)*split):])

    return trainloader, testloader, n_batches





# function to load images and format for ResNet (n_images, rgb, width, height)
def load_data(root_dir, max_count=1E10):
    
    # TODO: remove RGB EVENTUALLY; Find RESNET50 GREY
    # make array to load data from 4 classes
    data_loaded = np.zeros((0,3,200,200),'uint8')
    vals = []
    
    # LOAD MONOCROME DATA, USUALLY GREEN CHAN
    if False:
        for k in range(4):
            temp = np.repeat(np.load(root_dir+'/'+str(k)+'.npy')[None],3,axis=0).transpose(1,0,2,3)
            data_loaded = np.vstack((data_loaded,temp))
            vals.extend(np.zeros(temp.shape[0],'int32')+k)
    
    # LOAD RGB DATA (but NOTE THAT SECONDARY CHANS ARE messy)
    if False:
        for k in range(4):
            temp = np.load(root_dir+'/'+str(k)+'.npy').transpose(0,3,1,2)
            print (temp.shape)
            data_loaded = np.vstack((data_loaded,temp))
            vals.extend(np.zeros(temp.shape[0],'int32')+k)

    # LOAD RGB DATA, COPY GREEN CHAN TO EVERYTHING ELSE
    if True:
        green_chan = 1
        max_trials = max_count
        for k in range(4):
            temp = np.load(root_dir+'/'+str(k)+'.npy').transpose(0,3,1,2)[:,1]
            temp = np.repeat(temp[:,None],3,axis=1)
            
            if (temp[0,0]-temp[0,1]).sum()!=0:
                print ("BREAK ERROR")
                break
                
            idx = np.random.choice(np.arange(temp.shape[0]),
                                   max_trials,replace=False)
            
            temp = temp[idx]
            print (temp.shape)
            data_loaded = np.vstack((data_loaded,temp))
            vals.extend(np.zeros(temp.shape[0],'int32')+k)

    # convert lables to torch tensors
    vals = torch.tensor(vals, dtype=torch.long)

    # TRANSFORM DATA AS REQUIRED BY RESNET (?)
    train_data = []
    from tqdm import trange
    for k in range(vals.shape[0]):
        temp2 = train_transforms(data_loaded[k].transpose(1,2,0))
        train_data.append(temp2)  #THIS CAN BE DONE FASTER

    all_data = torch.stack(train_data)
    print ("Train data final [# samples, RGB, width, height]: ", all_data.shape)

    return all_data, vals
        

    
# same as above but for single images
def cnn_proceprocess_track_rgb(root_dir, 
                               save_formated_data=False):
        
    max_count = 1E10
    
    import glob
    
    # 
    track_id = os.path.split(root_dir)[1]
    fname = os.path.join(root_dir, str(track_id)+'.npz')
    #print ("fname: ", fname)

    #
    data = np.load(fname)
    data_loaded = data['images']
    frame_ids = data['frame_id']
    
#     data_loaded = [] #np.zeros((0,3,200,200),'uint8')
#     vals = []
#     frame_ids = []
#     for fname in fnames:
#         temp = np.load(fname)['frame']
        
#         if len(temp.shape)==2:
#             temp = np.repeat(temp[:,:,None],3,axis=2)
        
#         if temp.shape[0]!=200:
#             #print ('wrong size: ', temp.shape)
#             temp=temp[temp.shape[0]//2-100 : temp.shape[0]//2+100,
#                   temp.shape[0]//2-100 : temp.shape[0]//2+100]
            

#         data_loaded.append(temp)

#         #
#         frame_id = int(os.path.split(fname)[1].replace('frame_','')[:7])
#         #print (frame_id)
#         frame_ids.append(frame_id)
    
    #
    #frame_ids = np.array(frame_ids)
    
    
    # shuffle data; not sure this is needed;
    idx = np.random.choice(np.arange(data_loaded.shape[0]),
                           data_loaded.shape[0],
                           replace=False)

    data_loaded = data_loaded[idx]


    # save track id: 
    track_id = int(os.path.split(root_dir)[1])
    
    # convert lables to torch tensors
    train_data = []
    from tqdm import trange
    for k in range(data_loaded.shape[0]):
        #temp2 = train_transforms(data_loaded[k].transpose(1,2,0))
        temp2 = train_transforms(data_loaded[k])
        train_data.append(temp2)  #THIS CAN BE DONE FASTER

    all_data = torch.stack(train_data)
    #all_data = np.array(train_data)
    #print ("Train data final [# samples, RGB, width, height]: ", all_data.shape)

    #
    track_ids = np.zeros(frame_ids.shape[0],'int32')+track_id

        
    if save_formated_data:
        np.savez(fname_save,
            all_data = all_data,
            track_ids=track_ids,
            frame_ids = frame_ids)
        
    
    return all_data, track_id, frame_ids



    
# same as above but for single images
def cnn_proceprocess_directory(root_dir, 
                               save_formated_data=False):
    
    
    max_count = 1E10
    
    import glob
    
    # find all images in directory saved as .npz files each
    fnames = np.sort(glob.glob(root_dir + '/*.npz'))

    # LOAD RGB DATA, COPY GREEN CHAN TO EVERYTHING ELSE
    green_chan = 1
    max_trials = max_count
    data_loaded = [] #np.zeros((0,3,200,200),'uint8')
    vals = []
    frame_ids = []
    for fname in fnames:
        temp = np.load(fname)['frame']
        
        if len(temp.shape)==2:
            temp = np.repeat(temp[:,:,None],3,axis=2)
        
        if temp.shape[0]!=200:
            #print ('wrong size: ', temp.shape)
            temp=temp[temp.shape[0]//2-100 : temp.shape[0]//2+100,
                  temp.shape[0]//2-100 : temp.shape[0]//2+100]
            

        data_loaded.append(temp)

        #
        frame_id = int(os.path.split(fname)[1].replace('frame_','')[:7])
        #print (frame_id)
        frame_ids.append(frame_id)
    
    #
    frame_ids = np.array(frame_ids)
    
    
    # make stack of images
    data_loaded=np.array(data_loaded)
    #print ("data loaded: ", data_loaded.shape)
    # shuffle data; not sure this is needed;
    idx = np.random.choice(np.arange(data_loaded.shape[0]),
                           data_loaded.shape[0],replace=False)

    data_loaded = data_loaded[idx]


    # save track id: 
    track_id = int(os.path.split(root_dir)[1])
    
    # convert lables to torch tensors

    # TRANSFORM DATA AS REQUIRED BY RESNET (?)
    train_data = []
    from tqdm import trange
    for k in range(data_loaded.shape[0]):
        #temp2 = train_transforms(data_loaded[k].transpose(1,2,0))
        temp2 = train_transforms(data_loaded[k])
        train_data.append(temp2)  #THIS CAN BE DONE FASTER

    all_data = torch.stack(train_data)
    #all_data = np.array(train_data)
    #print ("Train data final [# samples, RGB, width, height]: ", all_data.shape)

    #
    track_ids = np.zeros(frame_ids.shape[0],'int32')+track_id

        
    if save_formated_data:
        np.savez(fname_save,
            all_data = all_data,
            track_ids=track_ids,
            frame_ids = frame_ids)
        
    
    return all_data, track_id, frame_ids

# DATA LOADER AND RANDOMIZER FUNCTION
def make_testloader(train_data, 
                    batch_size,
                    randomize=False):
    
    # RANDOMIZE DATA EVERYTIME THIS IS CALLED
    if randomize:
        idx = np.random.choice(np.arange(vals.shape[0]),
                         vals.shape[0],replace=False)
        # REARANGE DATA BASED ON RANDOMIZATION FLAG
        train_data = train_data[idx]

    # Compute number of batches
    n_batches = train_data.shape[0]//batch_size
    if (train_data.shape[0]/batch_size)!= train_data.shape[0]//batch_size:
        n_batches+=1

    # make test data
    data_predict = []
    for k in range(0,n_batches*batch_size,batch_size):
        data_predict.append(train_data[k:k+batch_size])

    # 
                      
    return data_predict, n_batches



def plot_bars(predictions, 
              confidence,
              test_data):
    
    clrs = ['red','blue','cyan','green']
    names = ['female','male','pup1 (shaved)','pup2 (unshaved)']

    import matplotlib.patches as mpatches
    import matplotlib.gridspec as gridspec

    plt.figure(figsize=(4, 4))
    G = gridspec.GridSpec(4, 4)

    # PLOT BAR GRAPHS FOR ALL PREDICTIONS
    axes_1 = plt.subplot(G[:1, 0])
    plt.title("All predicted labels")
    bins = np.arange(-0.5, 4.5, 1)
    y = np.histogram(predictions, bins = bins)
    for k in range(4):
        plt.bar(y[1][k], y[0][k], 0.9, color=clrs[k])
    
    # add legend
    handles, labels = axes_1.get_legend_handles_labels()
    for k in range(4):
        patch = mpatches.Patch(color=clrs[k], label=names[k])
        handles.append(patch) 
    plt.legend(handles=handles, loc='upper center')
    
    
    # PLOT BAR GRAPHS - THRESHOLD ON CONFIDENCe
    axes_1 = plt.subplot(G[1:2, 0])
    plt.title("Only high confidence labels")
    bins = np.arange(-0.5, 4.5, 1)
    
    threshold = 0.9
    idx_high_conf = np.where(confidence>threshold)[0]
    predictions_high_confidence = predictions[idx_high_conf]
    
    y_high_conf = np.histogram(predictions_high_confidence, bins = bins)
    for k in range(4):
        plt.bar(y_high_conf[1][k], y_high_conf[0][k], 0.9, color=clrs[k])
    
    # add legend
    handles, labels = axes_1.get_legend_handles_labels()
    for k in range(4):
        patch = mpatches.Patch(color=clrs[k], label=names[k])
        handles.append(patch) 
    plt.legend(handles=handles, loc='upper center')
    
    
    # MAKE IMAGE PLOTS
    max_id = np.argmax(y[0])
    print ("Main animal ", names[max_id])
    
    examples =[]
    example_ids = []
    for p in range(4):
        if p==max_id:
            continue
        example_ids.append(p)
        idx = np.where(predictions==p)[0]
        try:
            if idx.shape[0]>=3:
                frames = np.random.choice(idx, 3, replace=False)
            else:
                frames = np.random.choice(idx, 3)
        except:
            frames = [0,0,0]
            
        examples.append(frames)
    
    for k in range(3):
        ctr = 0
        frames = examples[k]
        for p in range(3):
            ax = plt.subplot(G[k,p+1])

            # get image
            temp = test_data[frames[ctr]].cpu().detach().numpy().transpose(1,2,0)
            plt.imshow(temp)

            plt.title("fr: "+str(frames[ctr])+ ", "+
                     names[predictions[frames[ctr]]])
            plt.xticks([])
            plt.yticks([])
            ctr+=1
            
            if p==0:
                plt.ylabel("examples \n"+str(names[example_ids[k]]))


    # PLOT TIME
    axes_2 = plt.subplot(G[3, :])
    clr_out = []
    for k in range(predictions.shape[0]):
        clr_out.append(clrs[predictions[k]])

    time = np.arange(predictions.shape[0])/25.
    plt.scatter(time, 
             np.ones(predictions.shape[0]),
             c=clr_out)
    
    # 
    clr_out = []
    for k in range(predictions_high_confidence.shape[0]):
        clr_out.append(clrs[predictions_high_confidence[k]])
        
    time_high_conf = idx_high_conf/25.
    plt.scatter(time_high_conf, 
             np.ones(predictions_high_confidence.shape[0])+1,
             c=clr_out)


    plt.xlabel("Time (sec)", fontsize=20)
    plt.tick_params(labelsize=20)
    plt.yticks([])
    plt.suptitle("CNN animal detected: "+names[max_id] + "(all frames) "
                 + str(round(np.max(y[0])/np.sum(y[0])*100,2))+"% of total track"
                 + "\nCNN animal detected (high confidence predictoin only): "+names[max_id] + " "
                 + str(round(np.max(y_high_conf[0])/np.sum(y_high_conf[0])*100,2))+"% of total track"
                 + "\n SLEAP tracklet # " + selected_track 
                 + " (# frames in track " 
                 +str(predictions.shape[0])+")", fontsize=18)
    plt.show()

# 
def load_training_data_run_prediction(fname_track, 
                                      model,
                                      device,
                                      recompute=False):
    
    max_count=1E10
    
    # grab track_id
    track_id = int(os.path.split(fname_track)[1])
    
    # prep save file;
    fname_out = os.path.join(fname_track,"predictions.npz")
    if os.path.exists(fname_out)==False:

        # preformat data
        #fname_formated = os.path.join(fname_track,"data_formated.npz")
        #if os.path.exists(fname_formated)==False:
        test_data, track_id, frame_ids = cnn_proceprocess_track_rgb(fname_track)
        #test_data = torch.from_numpy(all_data)
                
        # change model to evaluation mode to avoid batch normalization
        model.eval()

        # load the test data
        test_loader, n_batches = make_testloader(test_data, 
                                                  batch_size=500)

        predictions = []
        output_array = []
        for inputs in test_loader:
            # load to device
            inputs = inputs.to(device)

            n_trials = inputs.shape[0]

            # PREDICT;
            outputs = model(inputs)
            output_array.extend(outputs.cpu().detach().numpy())

            # get best predictions
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().detach().numpy())

        predictions = np.array(predictions)
        #print ("predictions: ", predictions.shape, predictions[:10])

        #probs = predictions 
        output_array = np.array(output_array)
        #print ("output array: ", output_array.shape)
        sig_pred = 1 / (np.exp(-output_array))  # confidence map

        # compute single predictions only
        if False:
            confidence = []
            for k in range(sig_pred.shape[0]):
                confidence.append(sig_pred[k][predictions[k]])
            confidence=np.array(confidence)
        else:
            confidence = sig_pred
        
        
        #print ("confidence; ", confidence[:10])
        
        np.savez(fname_out,
                 predictions=predictions,
                 confidence=confidence,
                 track_id=track_id,
                 frame_ids=frame_ids)
        
    else:
        data = np.load(fname_out)
        predictions = data['predictions']
        confidence = data['confidence']
        
    return predictions, confidence



def initialize_resnet():

    device = torch.device("cuda" if torch.cuda.is_available() 
                                      else "cpu")
    model = models.resnet50(pretrained=True)

    # Not sure what this does
    for param in model.parameters():
        param.requires_grad = False

    # Note sure what this does, effect on fc layer?
    model.fc = nn.Sequential(nn.Linear(2048, 512),
                                     nn.ReLU(),
                                     nn.Dropout(0.2),
                                     nn.Linear(512, 4),
                                     nn.LogSoftmax(dim=1))

    # todo: look up this loss
    criterion = nn.NLLLoss()

    # todo: look up this optimizer
    optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

    # move model to gpu
    model.to(device)

    
    return model, device, criterion, optimizer




In [3]:
###################################################
##### INIT MODEL AND LOAD SAVED MODEL (OPTIONAL) ##
###################################################
model, device, criterion, optimizer = initialize_resnet()

# load prev model params
if True:
    fname = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/cnn_training/model.pt'
    model.load_state_dict(torch.load(fname))
    #model.eval()

In [14]:
#########################################
######### PREDICT ON TEST DATA ##########
#########################################
root_dir = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/animal_images/'

selected_tracks = glob.glob(root_dir+"*")
print ("# tracks: ", len(selected_tracks))

from tqdm import tqdm
for selected_track in tqdm(selected_tracks):
    predictions, confidence = load_training_data_run_prediction(selected_track, 
                                                                model,
                                                                device)
    #print (np.unique(predictions, return_counts=True))
    

  0%|          | 0/2187 [00:00<?, ?it/s]

# tracks:  2187


  0%|          | 1/2187 [00:00<19:15,  1.89it/s]

(array([0, 1, 2, 3]), array([  2,  28, 106,  10]))
(array([2, 3]), array([3, 1]))


  0%|          | 3/2187 [00:01<17:37,  2.06it/s]

...

(array([0, 2, 3]), array([ 14, 153,   5]))
(array([1, 2]), array([ 1, 12]))
(array([1]), array([1]))
(array([1, 3]), array([3, 1]))
(array([2]), array([1]))


100%|█████████▉| 2183/2187 [16:47<00:00, 10.53it/s]

(array([2]), array([90]))
(array([1, 2, 3]), array([1, 1, 1]))
(array([1]), array([1]))
(array([2]), array([9]))
(array([2]), array([2]))
(array([3]), array([1]))


100%|██████████| 2187/2187 [16:47<00:00,  2.17it/s]

(array([1]), array([13]))
(array([1, 2]), array([3, 1]))


In [12]:
# 
fnames = [
'/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/animal_images/831/predictions.npz'
]

for fname in fnames:
    data = np.load(fname)

    confidence = data['confidence']
    print (confidence.shape)
    predictions = data['predictions']
    print (predictions[:100])
    print (np.unique(predictions, return_counts=True))

(146, 4)
[2 2 2 0 1 2 3 2 1 2 2 2 3 3 2 2 1 2 3 2 2 2 2 1 2 1 1 2 2 2 1 1 1 2 2 2 2
 1 2 3 2 2 3 2 2 2 3 2 2 2 2 2 2 2 2 1 3 2 2 2 2 3 2 2 1 2 2 2 2 2 2 2 2 2
 2 3 1 2 2 1 2 2 2 1 2 2 2 2 1 2 2 2 2 2 1 2 2 2 2 2]
(array([0, 1, 2, 3]), array([  2,  28, 106,  10]))


In [16]:
#######################################################################
######### COMBINE ALL PREDICTIONS INTO classificatoin_output.npz ######
#######################################################################
''' Combine all predictions into final file:
    

'''

def load_prediction_results(selected_track):

# prep save file;
    fname_out = os.path.join(selected_track,"predictions.npz")
    data = np.load(fname_out)
    
    preds = data['predictions']
    conf = data['confidence']
    frame_ids = data['frame_ids']
    track_id = data['track_id']

    track_ids = np.zeros(preds.shape[0])+track_id
    
    return track_ids, conf, frame_ids, preds
   
        
        
root_dir = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/animal_images/'

selected_tracks = glob.glob(root_dir+"*")
print ("# tracks: ", len(selected_tracks))

from tqdm import tqdm

vals = []
classes = []
frames_array = []
confidence = []
for selected_track in tqdm(selected_tracks):
    track_ids, conf_temp, frames_temp, preds = load_prediction_results(selected_track)
    
    vals.extend(track_ids)
    confidence.extend(conf_temp)
    frames_array.extend(frames_temp)
    classes.extend(preds)
    
# 
fname_out = os.path.split(root_dir[:-1])[0]+'/classification_output.npz'
print (fname_out)
np.savez(fname_out,
         vals = vals,
         classes = classes,
         confidence = confidence,
         frames_array = frames_array)


  2%|▏         | 54/2187 [00:00<00:04, 531.85it/s]

# tracks:  2187


100%|██████████| 2187/2187 [00:04<00:00, 539.80it/s]


/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/classification_output.npz


DONE

In [84]:
########################################
########### PLOT RESULTS ###############
########################################
    
plot_bars(predictions, confidence, test_data)



# #(ctr, ids, classes, logits) = predict_cnn2(x_test, y_test)
  
# classes = np.hstack(classes)
# print ("classes: ", classes)

# np.savez(root_dir+selected_output+'/classification_output.npz',
#          track_id=selected_track,
#          classes=predictions,
#          confidence=confidence,
#          )


# print ('done')
# REVIEW OUTPUT

# thresh = 0.8
# idx = np.where(confidence>thresh)[0]
# a, b = np.unique(predictions[idx], return_counts=True)
# if a.shape[0]>0:
#     print ("ids: ", a, " counts: ", b)
#     animal_best = a[np.argmax(b)]
#     highest = np.max(b)/predictions[idx].shape[0]
#     print ('Best animal predicted: ', animal_best, "  purity: ", highest, 
#            " # total frames: ", predictions.shape[0],
#           " # frames > threshold: ", idx.shape[0])
#     #print (predictions[:10])
# else:
#     print("no images over threhsold")

# for k in range(4):
#     ax=plt.subplot(2,2,k+1)
#     try:
#         idx = np.random.choice(np.arange(test_data.shape[0]))
#         plt.imshow(test_data[idx].squeeze().cpu().detach().numpy().transpose(1,2,0))
#     except:
#         pass
# plt.show()


Main animal  female


In [35]:
idx = np.where(c>0.9)[0]

print (np.unique(p[idx], return_counts=True))

(array([0, 2, 3]), array([ 1, 31, 47]))


In [60]:
data = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_9/2020-3-9_12_14_22_815059_compressed/cnn_output_10mins/0/frame_0000374_id_0.npz')
frame = data['frame']
print (frame.shape)

(200, 200)


In [17]:
data = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/animal_images/749/749.npz')

images  = data['images']
print (images.shape)

ctr=0
for k in range(0,172,10):
    ax=plt.subplot(2,5,ctr+1)
    plt.imshow(images[k])
    ctr+=1
plt.show()

(831, 200, 200, 3)


ValueError: num must be 1 <= num <= 10, not 11